# Limpieza del levantamiento de medicamentos en farmacias mexicanas y extranjeras

El objetivo es poder determinar si los resultados devueltos por las farmacias corresponden a los principios activos que efectivamente se necesitan para hacer la comparativa de precios. 
Para este caso, se realizarán dos pasos:
1. **Texto de la búsqueda contenido en el string producto**. Dado que el principio activo no es una variable que se pueda rescatar debido a la forma en que las farmacias registran los productos, muchas veces el principio puede venir contenida en la descripción del producto por lo que se buscará en el string completo.
2. **Búsqueda en COFEPRIS de marcas y nombres de producto**. Aquellos que no cumplen con el criterio del primer paso, se buscarán en la base de datos de COFEPRIS. Esto, para obtener las marcas de los principios activos. Si se obtiene un resultado, se utilizará el algoritmo Jaro Winckler para calcular la similitud entre la variable marca (COFEPRIS) y la variable product_name de nuestra búsqueda en farmacias. Aquellos que tengan un score mayor o igual a 0.8, se considerarán como válidos.

La finalidad de estos dos pasos es minimizar el número de medicamentos que se tendrán que revisar de manera manual.

Aquellos que no cumplieron ninguno de los dos criterios, deberán revisarse de manera manual para descartarlos o incluirlos en la base de datos.

In [1]:
#Librerías
import pandas as pd
from medscraper.buscadores import busca_cofepris
import textdistance
import time
import random

In [2]:
#Lectura de datos
url="https://raw.githubusercontent.com/labdatos-se/entregables_meds/main/piloto_seguimiento/levantamiento_21jul2023.csv"

In [3]:
sucia=pd.read_csv(url,encoding='latin')
sucia

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,20/07/2023 10:40,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0
2,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0
3,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0
4,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN


In [4]:
#Lectura de listado original para pegar el ATC
base=pd.read_excel("https://github.com/labdatos-se/entregables_meds/raw/main/piloto_seguimiento/presentaciones_146_pa.xlsx", sheet_name="Presentaciones")
base.columns=base.columns.str.lower()
#Renombrar primera columna
base.rename(columns={'principio activo':'pa'}, inplace=True)
#pa en minuscula
base['pa']=base['pa'].str.lower()
#Eliminar espacios en blanco
base['pa']=base['pa'].str.strip()
base['atc']=base['atc'].str.strip()
#Eliminar duplicados de pa
base.drop_duplicates(subset=['pa'], inplace=True)
#Dejar solo pa y atc 2
base=base[['pa','atc 2']]
base

c:\Users\claud\miniconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,pa,atc 2
0,acalabrutinib,L01
2,acarbosa,A10
6,ácido nicotinico,C10
8,adenosina,C01
9,afatinib,L01
...,...,...
591,venetoclax,L01
595,vildagliptina,A10
600,vinorelbina,L01
605,vismodegib,L01


In [5]:
#Pegar el ATC
sucia = pd.merge(sucia, base, left_on='busqueda', right_on='pa', how='left')
#Drop column pa
sucia.drop(columns=['pa'], inplace=True)
#rename atc 2 to atc
sucia.rename(columns={'atc 2':'atc'}, inplace=True)
sucia

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,20/07/2023 10:40,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0,L01
2,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01
3,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01
4,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0,C10
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN,C10
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN,C10
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN,C10


In [6]:
#Eliminar registros si son farmalisto_co, product_name janumet y busqueda sitagliptina
sucia=sucia[~((sucia['farmacia_corto']=='farmalisto_co') & (sucia['product_name']=='janumet') & (sucia['busqueda']=='sitagliptina'))]
sucia=sucia[~((sucia['farmacia_corto']=='farmalisto_co') & (sucia['product_name']=='glyxambi') & (sucia['busqueda']=='linagliptina'))]
#Búsquedas a eliminar
eliminar=["doxorrubicina","fenofibrato de colina","fenofibrato","hidroxicarbamida","temozolamida","vildagliptina"]
#Eliminar estos registros si son farmalisto_co
sucia=sucia[~((sucia['farmacia_corto']=='farmalisto_co') & (sucia['busqueda'].isin(eliminar)))]
sucia

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,20/07/2023 10:40,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0,L01
2,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01
3,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01
4,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0,C10
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN,C10
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN,C10
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN,C10


## Limpieza de datos
### Paso 1: La búsqueda está contenida en el string producto

In [7]:
#Hacer copia de la base de datos
draft=sucia.copy()

In [8]:
coincidencia = draft[draft.apply(lambda row: all(word in row['producto'] for word in row['busqueda'].split()), axis=1)]
#Si id de coincidencia es igual a id de la base de datos, entonces poner 1 en la columna coincidencia
draft['coincidencia'] = draft['id'].isin(coincidencia['id']).astype(int)
draft

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01,0
1,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,20/07/2023 10:40,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0,L01,1
2,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01,0
3,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01,0
4,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0,C10,0
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN,C10,0
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN,C10,0
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN,C10,0


In [9]:
#Se filtran aquellos que no coincidieron
nocoincide=draft[draft['coincidencia']==0]
nocoincide

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01,0
2,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01,0
3,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01,0
4,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01,0
5,11745,calcuencia,160121.99,20/07/2023 10:42,pchecker,acalabrutinib,canadianprescriptiondrugstore.com/,Canada,calcuencia,100.0,mg,QTY,56.0,L01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0,C10,0
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN,C10,0
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN,C10,0
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN,C10,0


### Cruce con COFEPRIS de marcas y nombres de producto

In [10]:
#hacer df única de busqueda
busqueda=nocoincide["busqueda"].unique()
busqueda=pd.DataFrame(busqueda, columns=['busqueda'])
busqueda

,busqueda
0,acalabrutinib
1,acarbosa
2,adenosina
3,afatinib
4,alogliptina
...,...
144,vildagliptina
145,vinorelbina
146,vismodegib
147,vorinostat


In [11]:
cofepris=pd.DataFrame()
for i in busqueda['busqueda']:
    for attempt in range(2):  # Máximo dos intentos
        try:
            cofepris=pd.concat([cofepris,busca_cofepris.registro(i)], ignore_index=True)
            # Dejar solo las columnas marca, den_gen y busqueda
            cofepris=cofepris[['marca','den_gen','busqueda']]
            break  # Si es exitoso, break este loop interno y pasar a la siguiente iteración del loop externo
        except Exception as e:
            print("Error en el medicamento", i)
            print("Detalles:", e)
            if attempt < 1:  # Sleep para el primer intento
                print("Esperando para reintentar...")
                time.sleep(60)  # Tiempo de espera para reintentar
    time.sleep(random.randint(1, 3))
cofepris


,marca,den_gen,busqueda
0,calquence,acalabrutinib,acalabrutinib
1,bosalvar,acarbosa,acarbosa
2,sacartrol,acarbosa,acarbosa
3,glucobay,acarbosa,acarbosa
4,sincrosa,acarbosa,acarbosa
...,...,...,...
328,setroxin,vinorelbina,vinorelbina
329,navelbine,vinorelbina,vinorelbina
330,erivedge,vismodegib,vismodegib
331,zolinza,vorinostat,vorinostat


In [12]:
#Hacer cruce de marca de cofepris con product_name de muestra
exacta=pd.merge(nocoincide, cofepris, left_on='busqueda', right_on='busqueda', how='left')
#Eliminar aquellos que tengan nan en marca
exacta=exacta[~exacta['marca'].isna()]
#Si marca de cofepris es igual a producto de muestra, entonces poner 1 en la columna coincidencia
#Comparar si marca y product_name son iguales
exacta['coincidencia'] = exacta['product_name']==exacta['marca']
#Convertir a entero
exacta['coincidencia'] = exacta['coincidencia'].astype(int)
#exacta['coincidencia'] = exacta['marca'].isin(exacta['product_name']).astype(int)
#Dejar los que son 1
exacta=exacta[exacta['coincidencia']==1]
exacta

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia,marca,den_gen
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib
1,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib
12,11753,calquence 100 mg tira de 6 capsulas paquete de 2,37297.22,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,calquence,100.0,mg,capsula,6.0,L01,1,calquence,acalabrutinib
22,11763,calquence,141235.71,20/07/2023 10:43,medicinpriser,acalabrutinib,medicinpriser,Dinamarca,calquence,100.0,mg,ampolleta,60.0,L01,1,calquence,acalabrutinib
23,11764,calquence,141235.71,20/07/2023 10:43,medicinpriser,acalabrutinib,medicinpriser,Dinamarca,calquence,100.0,mg,ampolleta,60.0,L01,1,calquence,acalabrutinib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,20872,erivedge,141994.21,20/07/2023 23:49,medicinpriser,vismodegib,medicinpriser,Dinamarca,erivedge,150.0,mg,NaN,28.0,L01,1,erivedge,vismodegib
22628,20873,erivedge,141412.62,20/07/2023 23:49,medicinpriser,vismodegib,medicinpriser,Dinamarca,erivedge,150.0,mg,NaN,28.0,L01,1,erivedge,vismodegib
22629,20874,erivedge 150 mg cap dura ct fr plas opc x 28,114480.52,20/07/2023 23:49,avisa,vismodegib,agencia nacional de vigilancia sanitaria brasil,Brasil,erivedge,150.0,mg,capsula,28.0,L01,1,erivedge,vismodegib
22631,20876,zolinza 100 mg caja con frasco con 120 capsulas,132083.00,20/07/2023 23:50,farmalisto_mx,vorinostat,farmalisto méxico,Mexico,zolinza,100.0,mg,capsula,120.0,L01,1,zolinza,vorinostat


In [13]:
#Hacer cruce de marca de cofepris con product_name de muestra. La coincidencia debe ser aproximada; es decir, si lo contiene
aprox=pd.merge(nocoincide, cofepris, left_on='busqueda', right_on='busqueda', how='left')
#Sustituir NaN por vacío
aprox=aprox[~aprox['marca'].isna()]
#Obtener distancia de levenshtein entre marca y producto
aprox['distancia']=aprox.apply(lambda row: textdistance.jaro_winkler(row['marca'], row['product_name']), axis=1)
#Si distancia es mayor a 0.8, entonces poner 1 en la columna coincidencia
aprox['coincidencia'] = aprox['distancia'].apply(lambda x: 1 if x >= 0.8 else 0)
#Dejar los que son 1
aprox=aprox[aprox['coincidencia']==1]
aprox

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia,marca,den_gen,distancia
0,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,1.000000
1,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,1.000000
2,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,0.811481
3,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01,1,calquence,acalabrutinib,0.811481
4,11745,calcuencia,160121.99,20/07/2023 10:42,pchecker,acalabrutinib,canadianprescriptiondrugstore.com/,Canada,calcuencia,100.0,mg,QTY,56.0,L01,1,calquence,acalabrutinib,0.811481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,20872,erivedge,141994.21,20/07/2023 23:49,medicinpriser,vismodegib,medicinpriser,Dinamarca,erivedge,150.0,mg,NaN,28.0,L01,1,erivedge,vismodegib,1.000000
22628,20873,erivedge,141412.62,20/07/2023 23:49,medicinpriser,vismodegib,medicinpriser,Dinamarca,erivedge,150.0,mg,NaN,28.0,L01,1,erivedge,vismodegib,1.000000
22629,20874,erivedge 150 mg cap dura ct fr plas opc x 28,114480.52,20/07/2023 23:49,avisa,vismodegib,agencia nacional de vigilancia sanitaria brasil,Brasil,erivedge,150.0,mg,capsula,28.0,L01,1,erivedge,vismodegib,1.000000
22631,20876,zolinza 100 mg caja con frasco con 120 capsulas,132083.00,20/07/2023 23:50,farmalisto_mx,vorinostat,farmalisto méxico,Mexico,zolinza,100.0,mg,capsula,120.0,L01,1,zolinza,vorinostat,1.000000


In [14]:
#Obtener registros con distancia mayor a 0.9 y menor a 1
x=aprox[(aprox['distancia']>0.95) & (aprox['distancia']<1)]
x

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia,marca,den_gen,distancia
255,12011,gilotrif,44047.80,20/07/2023 11:04,pchecker,afatinib,spfpharmacy.com,Canada,gilotrif,20.00,mg,tableta,28.0,L01,1,giotrif,afatinib,0.966667
256,12012,gilotrif,119731.84,20/07/2023 11:04,pchecker,afatinib,spfpharmacy.com,Canada,gilotrif,20.00,mg,tableta,84.0,L01,1,giotrif,afatinib,0.966667
283,12037,incresina p 25 mg30 mg caja con 28 tabletas,1193.12,20/07/2023 11:06,farmalisto_mx,alogliptina,farmalisto méxico,Mexico,incresina p,30.00,mg,tableta,28.0,A10,1,incresina,alogliptina,0.963636
287,12044,incresina 25 mg tab caja con 14,732.47,20/07/2023 11:07,fespecializadas,alogliptina,farmacias especializadas,Mexico,incresina,25.00,mg,tableta,14.0,A10,1,incresina p,alogliptina / pioglitazona,0.963636
290,12045,incresina 25 mg tab caja con 28,1135.51,20/07/2023 11:07,fespecializadas,alogliptina,farmacias especializadas,Mexico,incresina,25.00,mg,tableta,28.0,A10,1,incresina p,alogliptina / pioglitazona,0.963636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17442,19028,glitacar1 15mg tab caja con 7,271.79,20/07/2023 21:18,fespecializadas,pioglitazona,farmacias especializadas,Mexico,glitacar,15.00,mg,tableta,1.0,A10,1,glitacar-1,pioglitazona,0.960000
17447,19029,glitacar1 30mg tab caja con 30,1798.02,20/07/2023 21:18,fespecializadas,pioglitazona,farmacias especializadas,Mexico,glitacar,30.00,mg,tableta,1.0,A10,1,glitacar-1,pioglitazona,0.960000
17452,19030,glitacar1 30mg tab caja con 7,539.33,20/07/2023 21:18,fespecializadas,pioglitazona,farmacias especializadas,Mexico,glitacar,30.00,mg,tableta,1.0,A10,1,glitacar-1,pioglitazona,0.960000
17512,19043,glitacar 1.15mg 30 tabs,689.00,20/07/2023 21:18,farmacon,pioglitazona,farmacon,Mexico,glitacar,1.15,mg,tableta,30.0,A10,1,glitacar-1,pioglitazona,0.960000


In [15]:
#Pegar exacta y aprox
similitud=pd.concat([exacta,aprox], ignore_index=True)
#Si es el mismo id, dejar el que tiene distancia NaN
similitud=similitud.sort_values(by=['id','distancia'], ascending=[True, False]).drop_duplicates(subset=['id'], keep='first')
similitud

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia,marca,den_gen,distancia
1437,11740,calquence 100 mg caja con 56 capsulas,176000.00,20/07/2023 10:40,farmalisto_mx,acalabrutinib,farmalisto méxico,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,1.000000
1438,11742,calquence 100 mg cap caja con 56,186700.00,20/07/2023 10:40,fespecializadas,acalabrutinib,farmacias especializadas,Mexico,calquence,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,1.000000
1439,11743,calcuencia,150900.13,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,56.0,L01,1,calquence,acalabrutinib,0.811481
1440,11744,calcuencia,451778.56,20/07/2023 10:42,pchecker,acalabrutinib,canadianpharmacystore.com,Canada,calcuencia,100.0,mg,capsula,168.0,L01,1,calquence,acalabrutinib,0.811481
1441,11745,calcuencia,160121.99,20/07/2023 10:42,pchecker,acalabrutinib,canadianprescriptiondrugstore.com/,Canada,calcuencia,100.0,mg,QTY,56.0,L01,1,calquence,acalabrutinib,0.811481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100,20994,temozod 140 mg cap dura ct fr vd amb x 5,15420.94,21/07/2023 19:34,avisa,temozolomida,agencia nacional de vigilancia sanitaria brasil,Brasil,temozod,140.0,mg,capsula,5.0,L01,1,temozolomida,temozolomida,0.916667
4101,20995,temozod 250 mg cap dura ct fr vd amb x 5,27124.31,21/07/2023 19:34,avisa,temozolomida,agencia nacional de vigilancia sanitaria brasil,Brasil,temozod,250.0,mg,capsula,5.0,L01,1,temozolomida,temozolomida,0.916667
4102,20996,temodal 180mg con 5 cï¿½psulas envasado en sobr,24403.68,21/07/2023 19:34,araujo,temozolomida,farmacias araujo,Brasil,temodal,180.0,mg,sobre,5.0,L01,1,temozolomida,temozolomida,0.826190
4103,20997,temodal 250mg con 5 cï¿½psulas envasado en sobr,33385.67,21/07/2023 19:34,araujo,temozolomida,farmacias araujo,Brasil,temodal,250.0,mg,sobre,5.0,L01,1,temozolomida,temozolomida,0.826190


In [116]:
# Pegar primero y coincide
gral = pd.concat([coincidencia, similitud], ignore_index=True)

# Strip leading and trailing spaces
gral['product_name'] = gral['product_name'].str.strip()
gral['busqueda'] = gral['busqueda'].str.strip()

gral


,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia,marca,den_gen,distancia
0,11741,calquence 56 capsulas caja acalabrutinib 100.0 mg,184440.50,20/07/2023 10:40,fsanpablo,acalabrutinib,farmacias san pablo,Mexico,calquence,100.0,mg,capsula,56.0,L01,NaN,NaN,NaN,NaN
1,11769,acarbosa 50 mg caja con 30 tabletas,56.00,20/07/2023 10:45,farmalisto_mx,acarbosa,farmalisto méxico,Mexico,acarbosa,50.0,mg,tableta,30.0,A10,NaN,NaN,NaN,NaN
2,11770,acarbosa 100 mg caja con 30 tabletas,93.00,20/07/2023 10:45,farmalisto_mx,acarbosa,farmalisto méxico,Mexico,acarbosa,100.0,mg,tableta,30.0,A10,NaN,NaN,NaN,NaN
3,11772,acarbosa 50 mg alpharma 30 tabletas caja,80.00,20/07/2023 10:45,fsanpablo,acarbosa,farmacias san pablo,Mexico,acarbosa,50.0,mg,tableta,30.0,A10,NaN,NaN,NaN,NaN
4,11775,acarbosa 50mg 30 tabs,127.00,20/07/2023 10:45,farmacon,acarbosa,farmacon,Mexico,acarbosa,50.0,mg,tableta,30.0,A10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,20994,temozod 140 mg cap dura ct fr vd amb x 5,15420.94,21/07/2023 19:34,avisa,temozolomida,agencia nacional de vigilancia sanitaria brasil,Brasil,temozod,140.0,mg,capsula,5.0,L01,1.0,temozolomida,temozolomida,0.916667
3798,20995,temozod 250 mg cap dura ct fr vd amb x 5,27124.31,21/07/2023 19:34,avisa,temozolomida,agencia nacional de vigilancia sanitaria brasil,Brasil,temozod,250.0,mg,capsula,5.0,L01,1.0,temozolomida,temozolomida,0.916667
3799,20996,temodal 180mg con 5 cï¿½psulas envasado en sobr,24403.68,21/07/2023 19:34,araujo,temozolomida,farmacias araujo,Brasil,temodal,180.0,mg,sobre,5.0,L01,1.0,temozolomida,temozolomida,0.826190
3800,20997,temodal 250mg con 5 cï¿½psulas envasado en sobr,33385.67,21/07/2023 19:34,araujo,temozolomida,farmacias araujo,Brasil,temodal,250.0,mg,sobre,5.0,L01,1.0,temozolomida,temozolomida,0.826190


### Limpiezas particulares

In [117]:
alogliptina=gral[gral['busqueda']=='alogliptina']
len(alogliptina)

22

In [118]:
#Buscar en gral aquellos que tengan alogliptina en busqueda
alogliptina=gral[gral['busqueda']=='alogliptina']
#Dejar aquellos cuya den_gen sea igual a busqueda
alogliptina=alogliptina[alogliptina['den_gen']==alogliptina['busqueda']]
#Marcar como 1 en gral si es que el id está en alogliptina
gral['alogliptina'] = gral['id'].isin(alogliptina['id']).astype(int)
#Eliminar de gral aquellos que tengan alogliptina en busqueda y que sean cero en alogliptina
gral=gral[~((gral['busqueda']=='alogliptina') & (gral['alogliptina']==0))]
#Eliminar columna alogliptina
gral.drop(columns=['alogliptina'], inplace=True)

C:\Users\claud\AppData\Local\Temp\ipykernel_72360\2781721499.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gral.drop(columns=['alogliptina'], inplace=True)


In [136]:
#Obtener los ids que no coincidieron de acuerdo con la tabla nocoincide
buscar=nocoincide[~nocoincide['id'].isin(gral['id'])]
buscar

,id,producto,precio,hora_consulta,farmacia_corto,busqueda,farmacia,pais,product_name,quantity,unit,presentacion,num_prod,atc,coincidencia
14,11754,calaptin sr 120mg comprimido 15 s,15.77,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,calaptin sr,120.0,mg,tableta,15.0,L01,0
15,11755,tableta de lupinib 400mg,443.48,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,tableta de lupinib,400.0,mg,tableta,NaN,L01,0
16,11756,ibrunat 140mg frasco de 30 capsulas,1949.35,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,ibrunat,140.0,mg,capsula,30.0,L01,0
17,11757,cadinib 100mg tableta,754.12,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,cadinib,100.0,mg,tableta,NaN,L01,0
18,11758,pestana calcitab,28.73,20/07/2023 10:43,pharmeasy,acalabrutinib,pharmeasy,India,pestana calcitab,NaN,NaN,tableta,NaN,L01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276,11960,dtnfol 90 capsulas,446.45,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,dtnfol,NaN,NaN,capsula,90.0,C10,0
9277,11961,pomada propolis y miel propomel bio extratus b...,75.60,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,pomada propolis y miel propomel bio extratus b...,NaN,NaN,NaN,NaN,C10,0
9278,11962,crema para pies lisa pe 120g,41.99,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,crema para pies lisa pe,NaN,NaN,crema,NaN,C10,0
9279,11963,"crema para pies, manos y codos foot spa 90g",51.44,20/07/2023 10:55,araujo,ácido nicotinico,farmacias araujo,Brasil,"crema para pies, manos y codos foot spa",NaN,NaN,crema,NaN,C10,0


In [137]:
buscar = buscar[~((buscar['busqueda'] == 'alogliptina') & ((buscar['product_name'] == 'incresina duo') | (buscar['product_name'] == 'incresina p')))]

In [138]:
buscar["busqueda"].nunique() 

149

In [143]:
print("Registros a revisar: ",len(buscar))
print("Registros totales: ",len(buscar)+len(gral))
print("Porcentaje de registros a revisar: ",round(len(buscar)/(len(buscar)+len(gral))*100,2),"%")

Registros a revisar:  5471
Registros totales:  9259
Porcentaje de registros a revisar:  59.09 %


In [140]:
#Salvar gral y buscar como csv

gral.to_csv("C:/Users/claud/Documents/GitHub/entregables_meds/piloto_seguimiento/gral.csv", index=False, encoding='latin1')
buscar.to_csv("C:/Users/claud/Documents/GitHub/entregables_meds/piloto_seguimiento/buscar.csv", index=False, encoding='latin1')